In [1]:
# This script will read in the NetCDF files from a custom extraction (provincial) 
# and convert to CSV files for each grid cell

# This will clear all the variables before runnnig the code
for name in dir():
    if not name.startswith('_'):
        del globals()[name]

# Import required packages
import pandas as pd
import xarray as xr
import numpy as np
from netCDF4 import num2date

# Load in the NetCDF files for each RCP. This could be automated a bit better in terms of the 
# filenames and variable names, but works well for now.
var_rcp26_ds = xr.open_dataset('/Users/melamedturkishk/Desktop/Kai/Custom/QuebecBCCAQv2+ANUSPLIN300_ensemble-percentiles_historical+rcp26_1950-2100_gddgrow_0_YS.nc')
var_rcp45_ds = xr.open_dataset('/Users/melamedturkishk/Desktop/Kai/Custom/QuebecBCCAQv2+ANUSPLIN300_ensemble-percentiles_historical+rcp45_1950-2100_gddgrow_0_YS.nc')
var_rcp85_ds = xr.open_dataset('/Users/melamedturkishk/Desktop/Kai/Custom/QuebecBCCAQv2+ANUSPLIN300_ensemble-percentiles_historical+rcp85_1950-2100_gddgrow_0_YS.nc')

# Extract the time, lat, and lon data
time = np.asarray(var_rcp26_ds.time)
lat = np.asarray(var_rcp26_ds.lat)
lon = np.asarray(var_rcp26_ds.lon)

# Extract the 10th, 50th and 90th percentile data from each RCP. This can be something that's automated in
# a more slick way so you don't have to change the arrays we're extracting from, but for now this works fine
# RCP 2.6
var_p10_rcp26 = var_rcp26_ds.gddgrow_0_p10.data
var_p50_rcp26 = var_rcp26_ds.gddgrow_0_p50.data
var_p90_rcp26 = var_rcp26_ds.gddgrow_0_p90.data

# RCP 4.5
var_p10_rcp45 = var_rcp45_ds.gddgrow_0_p10.data
var_p50_rcp45 = var_rcp45_ds.gddgrow_0_p50.data
var_p90_rcp45 = var_rcp45_ds.gddgrow_0_p90.data

# RCP 8.5
var_p10_rcp85 = var_rcp85_ds.gddgrow_0_p10.data
var_p50_rcp85 = var_rcp85_ds.gddgrow_0_p50.data
var_p90_rcp85 = var_rcp85_ds.gddgrow_0_p90.data

# Extract the 10th, 50th, and 90th percentile time series for each grid point, concatenate together, then save as CSV
# Pre-allocate based on number of columns (10th percentile, 50th percentile, and 90th percentile for each RCP), 
# rows (years from 1950-2100) and lat and lon (3rd and 4th dimensions)
var_lat_lon_prcnt = np.zeros((var_p10_rcp26.shape[0], 9, var_p10_rcp26.shape[1], var_p10_rcp26.shape[2]))

# Pre-allocate a lat, lon variable that will provide the lat-lon coordinates corresponding to each grid point tme series
lat_lon_rep = np.zeros((var_p10_rcp26.shape[0], 2, var_p10_rcp26.shape[1], var_p10_rcp26.shape[2]))

for lt in range(var_p10_rcp26.shape[1]): 
    for ln in range(var_p10_rcp26.shape[2]):
        var_lat_lon_prcnt[:, :, lt, ln] = np.transpose([np.asarray(var_p10_rcp26[:, lt, ln]), 
                                                                    np.asarray(var_p50_rcp26[:, lt, ln]),
                                                                    np.asarray(var_p90_rcp26[:, lt, ln]), 
                                                                    np.asarray(var_p10_rcp45[:, lt, ln]), 
                                                                    np.asarray(var_p50_rcp45[:, lt, ln]),
                                                                    np.asarray(var_p90_rcp45[:, lt, ln]), 
                                                                    np.asarray(var_p10_rcp85[:, lt, ln]), 
                                                                    np.asarray(var_p50_rcp85[:, lt, ln]),
                                                                    np.asarray(var_p90_rcp85[:, lt, ln])])
        lat_lon_rep[:, :, lt, ln] = np.transpose(np.squeeze(np.asarray([np.asarray([np.repeat(lat[lt], var_p10_rcp26.shape[0])]), 
                                                  np.asarray([np.repeat(lon[ln], var_p10_rcp26.shape[0])])])))
        
# Convert to nparray
var_lat_lon_prcnt = np.asarray(var_lat_lon_prcnt)
lat_lon_rep = np.asarray(lat_lon_rep)

# Rearrange the dimensions and reshape the array so that the third dimension concatenates into the rows
var_lat_lon_prcnt_all = var_lat_lon_prcnt.transpose(3, 0, 1, 2).reshape(var_p10_rcp26.shape[0]*var_p10_rcp26.shape[2],
                                                                                                9, var_p10_rcp26.shape[1])
lat_lon_rep = lat_lon_rep.transpose(3, 0, 1, 2).reshape(var_p10_rcp26.shape[0]*var_p10_rcp26.shape[2],
                                                                                                2, var_p10_rcp26.shape[1])
#print(precip_tot_time_lat_lon_prcnt_all.shape)

# Repeat the same as above but for what was the fourth dimension, and is now the third
var_lat_lon_prcnt_all = var_lat_lon_prcnt_all.transpose(2, 0, 1).reshape(var_p10_rcp26.shape[0]*var_p10_rcp26.shape[2]*var_p10_rcp26.shape[1], 
                                                                                                 9)
lat_lon_rep = lat_lon_rep.transpose(2, 0, 1).reshape(var_p10_rcp26.shape[0]*var_p10_rcp26.shape[2]*var_p10_rcp26.shape[1], 
                                                                                                 2)

# Add in a column for the year that repeats for each grid point
year_rep = np.expand_dims(np.tile(np.arange(1950, 2101, 1), var_p10_rcp26.shape[1]*var_p10_rcp26.shape[2]), 
                          axis=1)

# Add/concatenate in the year, and the lat and lon to the actual time series data, for reference
var_time_lat_lon_prcnt_all = np.hstack((year_rep, lat_lon_rep, var_lat_lon_prcnt_all))

# Useful to see the final shape of the array you will save in order to figure out how many files to save
# Row limit in Excel: 1,048,576, Column limit in Excel: 16,384
#print(var_time_lat_lon_prcnt_all.shape)

# Finally, add in headers for each of the columns
headers = np.array(["Date", "Latitude", "Longitude", "RCP_2.6_10th_percentile", "RCP_2.6_50th_percentile", "RCP_2.6_90th_percentile", 
           "RCP_4.5_10th_percentile", "RCP_4.5_50th_percentile", "RCP_4.5_90th_percentile", "RCP_8.5_10th_percentile", 
           "RCP_8.5_50th_percentile", "RCP_4.5_90th_percentile"], dtype = object)

var_with_headers = np.vstack((headers, np.asarray(var_time_lat_lon_prcnt_all, dtype = object)))


C:\Users\melamedturkishk\AppData\Local\Continuum\anaconda3\lib\site-packages\xarray\core\merge.py:10: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [11]:
# Keeping this last bit of code in a separate cell because the precious cell takes longer to run, and can then
# simply run this last bit where we save the different CSV files multiple times until we get all the data
# This last bit of code could also probably still be automated in a better way, but for now, manually
# saving the data into ten separate CSV files because of row limit in Excel
# Save file as CSV with format (fmt) as string because of headers
np.savetxt("/Users/melamedturkishk/Desktop/Kai/Custom/deg_days_0_annual_ensemble_rcps_prcntiles_QC_10.csv", 
           np.vstack((headers, np.asarray(var_time_lat_lon_prcnt_all[1020005*9::, :], dtype = object))), delimiter = " ", fmt = "%s")